## Similarity Analysis

### Documents

In [9]:
doc1='Hello, my name is Himanshu'
doc2='Hi, my name is Himani'
string=[doc1,doc2]

### Preprocessing

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
doc1=word_tokenize(doc1)
doc2=word_tokenize(doc2)
sw = stopwords.words('english')
l1 =[];l2 =[]
X_set = {w for w in doc1 if not w in sw}
Y_set = {w for w in doc2 if not w in sw}
print(X_set)
print(Y_set)

{',', 'Himanshu', 'name', 'Hello'}
{'Hi', ',', 'Himani', 'name'}


### Creating Matrix

In [11]:
rvector = X_set.union(Y_set)
for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
c = 0
print(rvector)
print(l1)
print(l2)

{'Hi', 'name', 'Himani', ',', 'Himanshu', 'Hello'}
[0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 0, 0]


### Based on Cosine Similarity

In [12]:
import numpy as np
A=np.array(l1)
B=np.array(l2)
dot = np.dot(A,B)
norma = np.linalg.norm(A)
normb = np.linalg.norm(B)
cos = dot/(norma*normb)
print(cos)

0.5


### Based on Euclidean Distance

In [13]:
d = np.linalg.norm(A-B)
print(d)

2.0


## Sentiment Analysis

### Importing Data

In [14]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Preprocessing

In [15]:
from nltk.corpus import stopwords
data['review']=data['review'].str.lower()
sw = stopwords.words('english')
data['review']=data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

### Unigram Generation

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(data['review'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Splitting the dataset for train and test

In [17]:
from sklearn.model_selection import train_test_split
X=text_counts
y=data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=30)

### Fitting Multinomial Naive Bayes model

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
MNB = MultinomialNB()
MNB.fit(X_train, y_train)

predicted = MNB.predict(X_test)
accuracy_score = accuracy_score(predicted, y_test)

print('MultinominalNB model accuracy is',str('{:04.2f}'.format(accuracy_score*100))+'%')

MultinominalNB model accuracy is 86.10%


### Preprocessing for Neural Network models

In [19]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentiment_label=data['sentiment'].factorize()
review = data['review'].values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(review)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)
embedding_vector_length = 32

### Fitting RNN model

In [22]:
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential
RNN_model=Sequential()
RNN_model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
RNN_model.add(SimpleRNN(128, activation='tanh',return_sequences=False))
RNN_model.add(Dropout(0.2))
RNN_model.add(Dense(1, activation='sigmoid'))
RNN_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
RNN_model.fit(padded_sequence,sentiment_label[0],batch_size=64,epochs=10,verbose=1,validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 82s 127ms/step - loss: 0.5952 - accuracy: 0.6474 - val_loss: 0.3966 - val_accuracy: 0.8341
Epoch 2/10
625/625 [==============================] - 85s 137ms/step - loss: 0.3778 - accuracy: 0.8423 - val_loss: 0.3767 - val_accuracy: 0.8383
Epoch 3/10
625/625 [==============================] - 86s 138ms/step - loss: 0.3209 - accuracy: 0.8681 - val_loss: 0.3484 - val_accuracy: 0.8622
Epoch 4/10
625/625 [==============================] - 75s 120ms/step - loss: 0.3132 - accuracy: 0.8748 - val_loss: 0.4395 - val_accuracy: 0.7934
Epoch 5/10
625/625 [==============================] - 74s 119ms/step - loss: 0.4457 - accuracy: 0.7869 - val_loss: 0.6594 - val_accuracy: 0.5979
Epoch 6/10
625/625 [==============================] - 75s 119ms/step - loss: 0.3866 - accuracy: 0.8214 - val_loss: 0.3830 - val_accuracy: 0.8550
Epoch 7/10
625/625 [==============================] - 74s 118ms/step - loss: 0.2962 - accuracy: 0.8853 - val_loss: 0.3466 - val_ac

### Fitting LSTM model

In [24]:
from tensorflow.keras.layers import LSTM
LSTM_model=Sequential()
LSTM_model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
LSTM_model.add(LSTM(50, dropout=0.5,recurrent_dropout=0.5 ,activation='tanh',return_sequences=False))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(1, activation='sigmoid'))
LSTM_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(padded_sequence,sentiment_label[0],batch_size=64,epochs=5,verbose=1,validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 190s 301ms/step - loss: 0.3825 - accuracy: 0.8270 - val_loss: 0.2826 - val_accuracy: 0.8806
Epoch 2/5
625/625 [==============================] - 175s 281ms/step - loss: 0.2624 - accuracy: 0.8959 - val_loss: 0.2721 - val_accuracy: 0.8890
Epoch 3/5
625/625 [==============================] - 203s 324ms/step - loss: 0.2354 - accuracy: 0.9066 - val_loss: 0.3006 - val_accuracy: 0.8866
Epoch 4/5
625/625 [==============================] - 182s 291ms/step - loss: 0.2155 - accuracy: 0.9152 - val_loss: 0.3235 - val_accuracy: 0.8867
Epoch 5/5
625/625 [==============================] - 178s 286ms/step - loss: 0.2012 - accuracy: 0.9206 - val_loss: 0.3054 - val_accuracy: 0.8860


### Making Predictions

In [26]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(RNN_model.predict(tw).round().item())
    print("RNN Model Predicted label: ", sentiment_label[1][prediction])
    prediction = int(LSTM_model.predict(tw).round().item())
    print("LSTM Model Predicted label: ", sentiment_label[1][prediction])

import random
random_text=data['review'][random.randint(0, 5000)]
print(random_text)
predict_sentiment(random_text)

shot movie. proud film. great experience shows screen. halfdan hussey excellent collaborator vision able capture movie exact way envisioned prepping film. sets amazing well crafted character. john york team built sets fit characters, worked well shooting film, allowing us move seamlessly walls one set another. character amazing arc, makes great story. feel like actors gave excellent performances. disagree comments say acting good. watch decide yourself.
1/1 [==============================] - 0s 28ms/step
RNN Model Predicted label:  positive
1/1 [==============================] - 0s 36ms/step
LSTM Model Predicted label:  positive
